### check the [spglib](https://spglib.readthedocs.io/en/stable/) documentation

In [1]:
import itertools
import numpy as np
from numpy.linalg import norm
from ase import Atoms
from ase.spacegroup import crystal
from ase.spacegroup import Spacegroup
from ase.data import atomic_numbers, atomic_names
from ase.io import read
from ase.visualize import view
import spglib
import nglview as nv
from glob import glob

In [2]:
def view_structure(structure,myvec=[]):
    t = nv.ASEStructure(structure)
    w = nv.NGLWidget(t, gui=True)
    w.add_unitcell()
    w.add_ball_and_stick()
    w.add_representation('label',label_type='atomindex',color='black')
    w.add_representation('spacefill',selection=myvec,color="blue",radius=0.5)
    return w

### How can we check whether two crystals are equivalent or not?

<img src="crystal_equivalence.png" width=500 height=500>

### Function to obtain from spglib the symmetry informations of an ASE Atoms object

In [3]:
def get_sym_info(ase_atoms):
    return spglib.get_symmetry_dataset((ase_atoms.get_cell(),
                ase_atoms.get_scaled_positions(),
                ase_atoms.get_atomic_numbers()), 
                                   symprec=1e-5, angle_tolerance=-1.0)

### Let's create two ZrO2 crystals...

In [4]:
z1 = crystal('ZrO',[(0,0,0),(3./4.,1./4.,3./4.)],
             spacegroup=225,cellpar=[[5.09,0,0],[0,5.09,0],[0,0,5.09]]).repeat([2,1,1])

In [5]:
sym_ref = get_sym_info(z1)

In [6]:
z2=z1.copy()

### ...and in each crystal we raplace one Zr by one Hf

In [7]:
z1[0].symbol='Hf'
z2[3].symbol='Hf'
#z2[12].symbol='Hf'

### The two functions below allow to apply a crystal group operation (R+t) to an ASE Atoms

In [8]:
def apply_op(rotation,translation,ase_atoms):
    """ takes as input a rotation matrix a translation and an spg geometry
        and returns a np.array with 'atomtype,x,y,z' where x,y,z are the transformed coordinates of each atom
    """
    pnew=(np.matmul(ase_atoms.get_scaled_positions(),rotation.T) + translation) #% 1.0
    ase_atoms.set_scaled_positions(pnew)
def apply_invop(rotation,translation,ase_atoms):
    """ takes as input a rotation matrix a translation and an spg geometry
        and returns a np.array with 'atomtype,x,y,z' where x,y,z are the transformed coordinates of each atom
    """
    invrot=np.linalg.inv(rotation)
    pnew=(np.matmul(ase_atoms.get_scaled_positions(),invrot.T) -  np.matmul(invrot,translation))# % 1.0
    ase_atoms.set_scaled_positions(pnew)    

In [9]:
z1_ref=z1.copy()
apply_op(sym_ref['transformation_matrix'],sym_ref['origin_shift'],z1_ref)

### Applying to one crystal all possible symmetry operations we can check whetehr it is equivalent or not to the other one

In [10]:
#np.concatenate([np.array([1,2,3])[:, None], [[10,20,30],[100,200,300],[1000,2000,3000]]], axis=1)

In [11]:
def ase_a_equiv_ase_b(sym_ref,ase_a,ase_b):
    fullb = np.concatenate([ase_b.get_atomic_numbers()[:, None], (ase_b.get_scaled_positions())%1.0 ], axis=1)
    indices = np.lexsort((fullb[:, 3], fullb[:, 2], fullb[:, 1]))  # lexsort uses the last key as the primary sort key
    sorted_b = fullb[indices]
    ase_a_ref = ase_a.copy()
    for rotation,translation in zip(sym_ref['rotations'], sym_ref['translations']):
        new_ase = ase_a_ref.copy()
        apply_op(rotation,translation,new_ase)
        fulln = np.concatenate([new_ase.get_atomic_numbers()[:, None], (new_ase.get_scaled_positions())%1.0 ], axis=1)
        indices = np.lexsort((fulln[:, 3], fulln[:, 2], fulln[:, 1])) 
        sorted_n = fulln[indices]  
        if np.allclose(sorted_n, sorted_b, atol=1e-5):
            #ase_a.write('a.xyz')
            #ase_b.write('b.xyz')
            #new_ase.write('new.xyz')
            return True
    return False        

In [12]:
ase_a_equiv_ase_b(sym_ref,z1, z2)

True

### Now let's try to create several inequivalent structures where we replace O with N starting from a model structure

In [13]:
pristine = read('./quartz_alpha.xyz')
#atoms = read('./quartz_alpha_v2.xyz')
view_structure(pristine)

NGLWidget()

### Creation of the structures

In [14]:
all_structures=[]

In [15]:
oxygens = [atom.index for atom in pristine if atom.symbol == 'O']
print(len(oxygens))

96


In [16]:
nstructures = 50
nreplace = 2
for ns in range(nstructures):
    rho=[0,0]
    while rho[0] == rho[1]:
        rho=np.random.randint(0, high=len(oxygens), size=nreplace)
    new_geo = pristine.copy()
    elements =  new_geo.get_chemical_symbols()
    for i in rho:
        elements[oxygens[i]]='N'
    new_geo.set_chemical_symbols(elements)
    all_structures.append(new_geo)

### We check if there are equivalent structures

In [17]:
sym_ref=get_sym_info(pristine)
equivalent=[]
for i in range(nstructures):
    for j in range(i+1,nstructures):
        if ase_a_equiv_ase_b(sym_ref,all_structures[i], all_structures[j]):
            print(i,j)
            equivalent.append([i,j])

2 11
5 22
12 26
13 35
39 44


In [18]:
n=14
view_structure(all_structures[n])

NGLWidget()

In [19]:
n=19
view_structure(all_structures[n])

NGLWidget()

### Let's check if N-N distance in equivalent crystals is the same

In [20]:
def distance_Natoms(ase_geo):
    Natoms=[atom.index for atom in ase_geo if atom.symbol == 'N']
    return ase_geo.get_distance(Natoms[0],Natoms[1],mic=True)

In [21]:
for pair in equivalent:
    print(f"structures {pair[0]} and {pair[1]} N-N distance: {distance_Natoms(all_structures[pair[0]]):.3f} and {distance_Natoms(all_structures[pair[1]]):.3f}")

structures 2 and 11 N-N distance: 2.632 and 2.632
structures 5 and 22 N-N distance: 5.588 and 5.588
structures 12 and 26 N-N distance: 7.062 and 7.062
structures 13 and 35 N-N distance: 6.779 and 6.779
structures 39 and 44 N-N distance: 4.513 and 4.513
